In [1]:
import pandas as pd
import numpy as np
import numpy

In [2]:
num_unq = pd.read_csv('input_hdf/num_unq.csv')
cat_unq = pd.read_csv('input_hdf/cat_unq.csv')
date_unq = pd.read_csv('input_hdf/date_unq.csv')

In [5]:
date_train = pd.read_csv('input_orig/train_date.csv',usecols=date_unq.column.values, dtype=np.float32)

In [11]:
date_train.fillna(-999999,inplace=True)

In [12]:
#now, let's find features with no variance 
toKeep = []
toDelete = []
## loop through the DataFrame's columns
for col in date_train:
    ## if the value_counts method returns more than one uniqe entity,
    ## append the column name to 'toKeep'
    if len(date_train[col].value_counts()) > 1:
        toKeep.append(col)
    ## if not, append to 'toDelete'.
    else:
        toDelete.append(col)

In [13]:
date_train.drop(toDelete, 1, inplace = True)

In [14]:
date_train.shape

(1183747, 162)

In [15]:
corr_matrix = date_train.corr()
corr_matrix.to_csv('corr_matrix_date_unique.csv')
# substitude the entire matrix for a triangular matrix for faster
# computation
corr_matrix.ix[:,:] =  numpy.tril(corr_matrix.values, k = -1)
## create catcher objects to find lists of what is perfectly correlated
already_in = set()
result = []
for col in corr_matrix:
    perfect_corr = corr_matrix[col][abs(corr_matrix[col]) == 1.00].index.tolist()
    if perfect_corr and col not in already_in:
        already_in.update(set(perfect_corr))
        perfect_corr.append(col)
        result.append(perfect_corr)

toRemove = []
for item in result:
    toRemove.append(item[1:(len(item)+1)])
# flattenign the list of lists
toRemove = sum(toRemove, [])

In [16]:
date_train.drop(toRemove, 1, inplace = True)

In [17]:
date_train.shape

(1183747, 162)

In [18]:
date_train.to_hdf('train_date_162cols.hdf5','table')

In [19]:
date_test = pd.read_csv('input_orig/test_date.csv', usecols=date_train.columns.values, dtype=np.float32)

In [20]:
date_train.fillna(-999999,inplace=True)

In [21]:
date_test.shape

(1183748, 162)

In [22]:
date_test.to_hdf('test_date_162cols.hdf5','table')